[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/NirDiamant/RAG_Techniques/blob/main/all_rag_techniques/simple_rag.ipynb)

# 简单RAG（检索增强生成）系统

## 概述

此代码实现了一个基本的检索增强生成（RAG）系统，用于处理和查询PDF文档。该系统将文档内容编码到向量存储中，然后可以查询以检索相关信息。

## 核心组件

1. PDF处理和文本提取
2. 文本分块以便于处理
3. 使用[FAISS](https://engineering.fb.com/2017/03/29/data-infrastructure/faiss-a-library-for-efficient-similarity-search/)和OpenAI嵌入创建向量存储
4. 设置检索器以查询处理后的文档
5. RAG系统评估

## 方法详情

### 文档预处理

1. 使用PyPDFLoader加载PDF。
2. 使用RecursiveCharacterTextSplitter将文本分割成指定大小和重叠的块。

### 文本清理

应用自定义函数`replace_t_with_space`来清理文本块。这可能解决PDF中的特定格式问题。

### 向量存储创建

1. 使用OpenAI嵌入创建文本块的向量表示。
2. 从这些嵌入创建FAISS向量存储，以实现高效的相似性搜索。

### 检索器设置

1. 配置检索器以获取给定查询的前2个最相关的块。

### 编码函数

`encode_pdf`函数封装了加载、分块、清理和将PDF编码到向量存储的整个过程。

## 主要特性

1. 模块化设计：编码过程封装在单个函数中，便于重用。
2. 可配置分块：允许调整块大小和重叠。
3. 高效检索：使用FAISS进行快速相似性搜索。
4. 评估：包含评估RAG系统性能的函数。

## 使用示例

代码包含一个测试查询："气候变化的主要原因是什么？"。这演示了如何使用检索器从处理后的文档中获取相关上下文。

## 评估

系统包含一个`evaluate_rag`函数来评估检索器的性能，尽管提供的代码中没有详细说明使用的具体指标。

## 此方法的优势

1. 可扩展性：通过分块处理可以处理大型文档。
2. 灵活性：易于调整块大小和检索结果数量等参数。
3. 效率：利用FAISS在高维空间中进行快速相似性搜索。
4. 与先进NLP集成：使用OpenAI嵌入进行最先进的文本表示。

## 结论

这个简单的RAG系统为构建更复杂的信息检索和问答系统提供了坚实的基础。通过将文档内容编码到可搜索的向量存储中，它能够高效地检索相关信息以响应查询。这种方法对于需要快速访问大型文档或文档集合中特定信息的应用程序特别有用。

# 软件包安装和导入

下面的单元格安装了运行此笔记本所需的所有必要软件包。


In [23]:
# Install required packages
!pip install pypdf==5.6.0
!pip install PyMuPDF==1.26.1
!pip install python-dotenv==1.1.0
!pip install langchain-community==0.3.25
!pip install langchain_openai==0.3.23
!pip install rank_bm25==0.2.2
!pip install faiss-cpu==1.11.0
!pip install deepeval==3.1.0

In [7]:
# Clone the repository to access helper functions and evaluation modules
!git clone https://github.com/NirDiamant/RAG_TECHNIQUES.git
import sys
sys.path.append('RAG_TECHNIQUES')

# If you need to run with the latest data
# !cp -r RAG_TECHNIQUES/data .

Cloning into 'RAG_TECHNIQUES'...
remote: Enumerating objects: 1531, done.
remote: Counting objects: 100% (808/808), done.
remote: Compressing objects: 100% (359/359), done.
remote: Total 1531 (delta 549), reused 458 (delta 449), pack-reused 723 (from 2)
Receiving objects: 100% (1531/1531), 34.20 MiB | 25.58 MiB/s, done.
Resolving deltas: 100% (962/962), done.


In [2]:
import os
import sys
from dotenv import load_dotenv
from google.colab import userdata



# Load environment variables from a .env file
load_dotenv()

# Set the OpenAI API key environment variable (comment out if not using OpenAI)
if not userdata.get('OPENAI_API_KEY'):
    os.environ["OPENAI_API_KEY"] = input("Please enter your OpenAI API key: ")
else:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Original path append replaced for Colab compatibility

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from helper_functions import (EmbeddingProvider,
                              retrieve_context_per_question,
                              replace_t_with_space,
                              get_langchain_embedding_provider,
                              show_context)

from evaluation.evalute_rag import evaluate_rag

from langchain.vectorstores import FAISS


<ipython-input-2-622825101>:26: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from helper_functions import (EmbeddingProvider,


### Read Docs

In [3]:
# 下载所需的数据文件
import os
os.makedirs('data', exist_ok=True)

# 下载本笔记本中使用的PDF文档
!wget -O data/Understanding_Climate_Change.pdf https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/Understanding_Climate_Change.pdf
!wget -O data/Understanding_Climate_Change.pdf https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/Understanding_Climate_Change.pdf


--2025-06-14 07:31:48--  https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/Understanding_Climate_Change.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 206372 (202K) [application/octet-stream]
Saving to: ‘data/Understanding_Climate_Change.pdf’

data/Understanding_ 100%[===================>] 201.54K  --.-KB/s    in 0.03s   

2025-06-14 07:31:48 (5.89 MB/s) - ‘data/Understanding_Climate_Change.pdf’ saved [206372/206372]

--2025-06-14 07:31:48--  https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/Understanding_Climate_Change.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185

In [4]:
path = "data/Understanding_Climate_Change.pdf"

### 编码文档

In [5]:
def encode_pdf(path, chunk_size=1000, chunk_overlap=200):
    """
    使用OpenAI嵌入将PDF书籍编码到向量存储中。

    Args:
        path: PDF文件的路径。
        chunk_size: 每个文本块的期望大小。
        chunk_overlap: 连续块之间的重叠量。

    Returns:
        包含编码书籍内容的FAISS向量存储。
    """

    # 加载PDF文档
    loader = PyPDFLoader(path)
    documents = loader.load()

    # 将文档分割成块
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap, length_function=len
    )
    texts = text_splitter.split_documents(documents)
    cleaned_texts = replace_t_with_space(texts)

    # 创建嵌入（已测试OpenAI和Amazon Bedrock）
    embeddings = get_langchain_embedding_provider(EmbeddingProvider.OPENAI)
    #embeddings = get_langchain_embedding_provider(EmbeddingProvider.AMAZON_BEDROCK)

    # 创建向量存储
    vectorstore = FAISS.from_documents(cleaned_texts, embeddings)

    return vectorstore

In [6]:
chunks_vector_store = encode_pdf(path, chunk_size=1000, chunk_overlap=200)

### 创建检索器

In [7]:
chunks_query_retriever = chunks_vector_store.as_retriever(search_kwargs={"k": 2})

### 测试检索器

In [8]:
test_query = "气候变化的主要原因是什么？"
context = retrieve_context_per_question(test_query, chunks_query_retriever)
show_context(context)

Context 1:
Chapter 2: Causes of Climate Change 
Greenhouse Gases 
The primary cause of recent climate change is the increase in greenhouse gases in the 
atmosphere. Greenhouse gases, such as carbon dioxide (CO2), methane (CH4), and nitrous 
oxide (N2O), trap heat from the sun, creating a "greenhouse effect." This effect is essential 
for life on Earth, as it keeps the planet warm enough to support life. However, human 
activities have intensified this natural process, leading to a warmer climate. 
Fossil Fuels 
Burning fossil fuels for energy releases large amounts of CO2. This includes coal, oil, and 
natural gas used for electricity, heating, and transportation. The industrial revolution marked 
the beginning of a significant increase in fossil fuel consumption, which continues to rise 
today. 
Coal


Context 2:
Most of these climate changes are attributed to very small variations in Earth's orbit that 
change the amount of solar energy our planet receives. During the Holocene epoch,

/content/RAG_TECHNIQUES/helper_functions.py:143: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = chunks_query_retriever.get_relevant_documents(question)


### 评估结果

In [9]:
# 注意 - 目前仅适用于OPENAI
evaluate_rag(chunks_query_retriever)

{'questions': ['1. **Multiple Choice: Causes of Climate Change**',
  '   - What is the primary cause of the current climate change trend?',
  '     A) Solar radiation variations',
  '     B) Natural cycles of the Earth',
  '     C) Human activities, such as burning fossil fuels',
  '     D) Volcanic eruptions',
  '',
  '2. **True or False: Climate Change Impacts**',
  '   - True or False: Climate change only affects the temperature of the planet, not weather patterns, sea levels, or ecosystems.',
  '',
  '3. **Short Answer: Mitigation Strategies**',
  '   - Describe two effective strategies that could be implemented to mitigate the effects of climate change.',
  '',
  '4. **Matching: Climate Change Terminology**',
  '   - Match the following terms with their correct definitions:',
  '     A) Greenhouse Gases',
  '     B) Carbon Footprint',
  '     C) Renewable Energy',
  '     D) Adaptation',
  '     - Definitions:',
  '       1. The total amount of greenhouse gases produced to directl